<h3>importing libraries</h3>

In [146]:
from sklearn.model_selection import train_test_split
import pandas as pd
from scipy.stats import zscore
import numpy as np
from sklearn.preprocessing import StandardScaler

<h5>read data and explore it </h5>

In [147]:
data=pd.read_csv("Your_data.csv")
data.head(3)
data.info()
print(str(data.isnull().sum()))
print(data.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159256 entries, 0 to 159255
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unnamed: 0      159256 non-null  int64  
 1   Cholesterol     159256 non-null  int64  
 2   triglyceride    159256 non-null  int64  
 3   age             159256 non-null  int64  
 4   hearing(right)  159256 non-null  int64  
 5   hemoglobin      159256 non-null  float64
 6   LDL             159256 non-null  int64  
 7   dental caries   159256 non-null  int64  
 8   systolic        159256 non-null  int64  
 9   hearing(left)   159256 non-null  int64  
 10  eyesight(left)  159256 non-null  float64
 11  smoking         159256 non-null  int64  
dtypes: float64(2), int64(10)
memory usage: 14.6 MB
Unnamed: 0        0
Cholesterol       0
triglyceride      0
age               0
hearing(right)    0
hemoglobin        0
LDL               0
dental caries     0
systolic          0
hearing(left)

<h4> data split in train , test and validataion</h4>

In [148]:
data.drop(data.columns[0], axis=1,inplace=True)
train_data, tmp_data = train_test_split(data, test_size=0.3, random_state=42, stratify=data['smoking'])
test_data,validate_data=train_test_split(tmp_data, test_size=0.5, random_state=42)
print(train_data.shape)

(111479, 11)


<h3> remove outliers from train data by percentile</h3>
<p> test age feature</p>

In [149]:

max_threshold=train_data['age'].quantile(1)
min_threshold=train_data['age'].quantile(0)
print(max_threshold,min_threshold)
#all ages are possible ages of smokers won't remove any

85.0 20.0


<p> test LDL feature</p>

In [150]:
max_threshold = train_data['LDL'].quantile(0.999)
min_threshold = train_data['LDL'].quantile(0.001)
print(max_threshold,min_threshold)
train_data = train_data[(train_data['LDL'] < max_threshold) & (train_data['LDL'] > min_threshold)]

print(train_data.shape)

202.0 42.0
(111249, 11)


<p> test chelosterol feature</p>

In [151]:
max_threshold = train_data['Cholesterol'].quantile(0.999)
min_threshold = train_data['Cholesterol'].quantile(0.001)
print(max_threshold,min_threshold)
train_data = train_data[(train_data['Cholesterol'] < max_threshold) & (train_data['Cholesterol'] > min_threshold)]

print(train_data.shape)

283.0 118.0
(111015, 11)


<p>test systolic feature</p>

In [152]:
max_threshold = train_data['systolic'].quantile(0.999)
min_threshold = train_data['systolic'].quantile(0.001)
print(max_threshold,min_threshold)
train_data = train_data[(train_data['systolic'] < max_threshold) & (train_data['systolic'] > min_threshold)]

print(train_data.shape)

170.0 90.0
(110352, 11)


<h3>z score normalization to remove outliers</h3>
<p>  is suited for triglyceride distribution</p>

In [153]:

z_scores = zscore(train_data['triglyceride'])
z_score_threshold = 3
outliers = np.abs(z_scores) > z_score_threshold

train_data = train_data[~outliers]
print(train_data.shape)

(109205, 11)


In [154]:
#hemoglobin
z_scores = zscore(train_data['hemoglobin'])
z_score_threshold = 3
outliers = np.abs(z_scores) > z_score_threshold

train_data = train_data[~outliers]
print(train_data.shape)

(108399, 11)


In [155]:
#eyesight(left)
z_scores = zscore(train_data['eyesight(left)'])
z_score_threshold = 3
outliers = np.abs(z_scores) > z_score_threshold

train_data = train_data[~outliers]
print(train_data.shape)

(108306, 11)


<h3>splitting train,test,validate data into x and y </h3>

In [156]:

X_train=train_data.drop(['smoking'],axis=1)
Y_train=train_data['smoking']
X_test=test_data.drop(['smoking'],axis=1)
Y_test=test_data['smoking']
X_validate=validate_data.drop(['smoking'],axis=1)
Y_validate=validate_data['smoking']

<h3>data normalization</h3>

In [158]:
normalizer=StandardScaler()
X_train=normalizer.fit_transform(X_train)
X_validate=normalizer.transform(X_validate)
X_test = normalizer.transform(X_test)